In [49]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame
from risk_metrics import Risk_Metrics
import os
import logging

In [50]:
connection = create_connection("../database/crypto_billionairs.db")

In [51]:
headers = ["time", "open", "high", "low", "close", "volume"]

In [52]:
def momentum(df, lag):
    return df.pct_change(periods=lag)

In [59]:
def prepare_files(path, db_connection, header_list):
    
    for file in os.listdir(path):
            
            if file[-3:] == 'txt':
                file_name = str(file).replace("-", "_")
                df = pd.read_csv(f'./{path}/{file}', names = header_list)
                
                df.to_sql(f'{file_name[:-4]}_complete_raw', con=db_connection, if_exists="replace", index=False)
             
                df["momentum_1week"] = momentum(df["close"], 10080)
                
                df['time_str'] = df['time']
                df['time'] = pd.to_datetime(df['time'])
                
                df = df.set_index('time')
                
                df = df.resample('1T').mean().interpolate()
                df['time'] = df.index
                df["year"] = df["time"].astype(str).str[:4]
                df["month"] = df["time"].astype(str).str[5:7]
                df["day"] = df["time"].astype(str).str[8:10]
                
                

                df_ti = pd.DataFrame()
                df_ti["time"] = df["time"]
                df_ti["open"] = df["open"]
                df_ti["close"] = df["close"]
                df_ti["high"] = df["high"]
                df_ti["low"] = df["low"]
                df_ti["volume"] = df["volume"]
                
                stock = StockDataFrame.retype(df_ti)
                df["stochastic_oscillator"] = stock.get("kdjk")
                df["relative_strength_index"] = stock.get("rsi_30")
                df["moving_average_convergence_divergence"] = stock.get("macd")
                df["moving_average_convergence_divergence"] = df["moving_average_convergence_divergence"].astype(int)
                df["so_group"] = df["stochastic_oscillator"].astype(str).str[0]
                df["rsi_group"] = df["relative_strength_index"].astype(str).str[0]
                df["technical_indicators"] = df["so_group"] + "/" + df["rsi_group"]
                df.to_sql(f'{file_name[:-4]}_complete_history_preprocessed', con=db_connection, if_exists="replace", index=False)

                df["buy_indicator"] = 0
                df.loc[df["technical_indicators"] == "1/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "2/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "3/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "4/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "5/7", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "6/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "8/1", 'buy_indicator'] = 1
                
                df["close_indicator"] = 0
                df.loc[df["technical_indicators"].shift(10080) == "1/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "2/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "3/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "4/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "5/7", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "6/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "8/1", 'close_indicator'] = 2
                
                
                short_signals = df.index[df["moving_average_convergence_divergence"] >= 55].tolist()
                df["short_indicator"] = 0
                df["short_indicator"].loc[short_signals] = 1
            
                
                df.to_sql(f'{file_name[:-4]}_complete_history_long_short_rsi', con=db_connection, if_exists="replace", index=False)
                #os.remove(f'{path}/{file}')
        

In [60]:
prepare_files("../database/", connection, headers)

In [55]:
class Momentum_RSI_long_short(Strategy):
   
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.buy_init = self.I(init_buy_signal, self.data.df)
        self.close_buy_init = self.I(init_close_long_signal, self.data.df)
        self.short_init = self.I(init_short_signal, self.data.df)
        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.buy_init[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif self.close_buy_init[-1] == 2 and self.position.is_long:
             self.position.close()
             
        elif self.short_init[-1] == -1 and self.position.is_short is False:
            self.position.close()
            self.sell()

In [56]:
def init_buy_signal(trash):
    return df_temp['buy_indicator']

def init_short_signal(trash):
    return df_temp['short_indicator']

def init_close_long_signal(df):
    return df_temp['close_indicator']

In [57]:
def run_backtesting(db_connection):
    
    global df_temp
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "1min_complete_history_long_short_rsi" in name and 'trades' not in name]
    print(filtered_table_names)
    df_risk = pd.DataFrame(columns = range(10))
    df_risk.columns = ["table_name", "timehorizon", "return", "annualized_return", "sharpe_ratio_annualized", "sortino_ratio_annualized", "maximum_drawdown", "calmar_ratio_annualized", "trades_count", "win_rate"]
    
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
        
        df_backtesting = pd.DataFrame()
        df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_temp[['open', 'high', 'low', 'close', 'volume']]
        
        
        bt = Backtest(df_backtesting, Momentum_RSI_long_short, cash=100_000, commission=.001)
        stats = bt.run()
        trades = pd.DataFrame(stats['_trades'])
        trades.to_sql(f"trades_{table}", db_connection, if_exists="replace")
        
        risk = Risk_Metrics(trades, df_temp[10080:], 0)
        risk_metrics_list = [f"{table}", len(df_backtesting), stats["Return [%]"], risk._annualize(stats["Return [%]"]),
                             risk.sharpe_ratio("momentum_1week"), risk.sortino_ratio("momentum_1week"),
                             risk.max_drawdown(), risk.calmar_ratio(stats["Return [%]"]), len(trades), stats["Win Rate [%]"]]
        print(risk_metrics_list)
        #df_risk.append(risk_metrics_list)
        df_risk.loc[len(df_risk)] = risk_metrics_list
        
    
    df_risk.to_sql("cryptocurrencies_risk_metrics_1m_rsi", db_connection, if_exists="replace")

In [58]:
run_backtesting(connection)

['BTC_1min_complete_history_long_short_rsi', 'DOGE_1min_complete_history_long_short_rsi', 'ETH_1min_complete_history_long_short_rsi']


<ipython-input-57-769d60451720>:22: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, Momentum_RSI_long_short, cash=100_000, commission=.001)


['BTC_1min_complete_history_long_short_rsi', 4734689, 487.27126964684635, 54.20761365149633, nan, 4.983118900216542, 0.8258539760947463, 65.63825472855427, 2131, 40.73205068043172]
['DOGE_1min_complete_history_long_short_rsi', 2541831, 73.1544149130391, 15.187101922066331, nan, 1.7959858038742451, 0.8701626668212298, 17.45317571201482, 1559, 40.089801154586276]
['ETH_1min_complete_history_long_short_rsi', 3186368, -9.440112395313786, -1.562113723622331, nan, 0.8406501243423516, 0.07476613829489084, -20.893331650500375, 2162, 36.35522664199815]
